# Resultados PROJECOES

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# ------------------------------------------------------------------------------
# Script: Consolidar Vazões Simuladas por Sub-bacia (MGB + CLIMBra)
# ------------------------------------------------------------------------------
# Este script percorre a pasta com resultados simulados do MGB para 19 GCMs 
# em 9 sub-bacias, consolida os arquivos de saída (SIM_MC_*.TXT) e organiza os 
# dados por sub-bacia.
#
# Para cada sub-bacia, é criado um único arquivo .csv contendo a série temporal 
# diária de 2015 a 2100, com colunas para cada GCM, além das colunas de data: 
# 'dia', 'mes' e 'ano'. O nome do arquivo inclui o código e o nome da estação.
#
# Observação: Os arquivos simulados foram originalmente gerados com datas 
# regredidas em 80 anos (ex: 2020 → 1940). Este script adiciona novamente os 
# 80 anos à coluna 'ano', restaurando a data original.
# ------------------------------------------------------------------------------

import os
import pandas as pd
from pathlib import Path
from collections import defaultdict

# === Caminhos ===
PASTA_ENTRADA = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima")
PASTA_SAIDA   = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245")
PASTA_SAIDA.mkdir(exist_ok=True)

PASTA_SAIDA_DIARIOS = PASTA_SAIDA / "diarios"
PASTA_SAIDA_DIARIOS.mkdir(parents=True, exist_ok=True)

# === Caminho do arquivo de mapeamento ===
CAMINHO_MAPEAMENTO = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_mini.csv")

# === Leitura do mapeamento código_mini → nome_estacao ===
df_mapeamento = pd.read_csv(CAMINHO_MAPEAMENTO, sep=";")

# === Dicionário onde cada sub-bacia conterá os dados dos GCMs ===
dados_subbacias = defaultdict(list)

# === Percorrer todos os modelos ===
for modelo_path in PASTA_ENTRADA.iterdir():
    if not modelo_path.is_dir():
        continue

    nome_modelo = modelo_path.name
    output_path = modelo_path / "Output"

    if not output_path.exists():
        continue

    for arquivo in output_path.glob("SIM_MC_*.TXT"):
        subbacia = arquivo.stem.split("_")[-1]  # ex: '792'

        try:
            df = pd.read_csv(arquivo, delim_whitespace=True, names=["dia", "mes", "ano", nome_modelo])
        except Exception as e:
            print(f"❌ Erro ao ler {arquivo.name}: {e}")
            continue

        # Corrige o ano regredido
        df["ano"] = df["ano"] + 80

        dados_subbacias[subbacia].append(df)

# === Para cada sub-bacia, consolidar e salvar ===
for subbacia, lista_df in dados_subbacias.items():
    df_base = lista_df[0][["dia", "mes", "ano"]].copy()

    for df in lista_df:
        nome_coluna = df.columns[-1]
        df_base[nome_coluna] = df[nome_coluna].values

    # Obter nome da estação (se existir no mapeamento)
    linha = df_mapeamento[df_mapeamento["codigo_mini"].astype(str) == subbacia]

    if not linha.empty:
        codigo_estacao = str(linha["estacao_obs"].values[0])
        nome_estacao   = linha["nome_estacao"].values[0]
    else:
        codigo_estacao = f"SUB{subbacia}"
        nome_estacao   = "Desconhecida"

    nome_arquivo = f"{codigo_estacao}.csv"
    caminho_saida = PASTA_SAIDA_DIARIOS / nome_arquivo
    df_base.to_csv(caminho_saida, index=False, sep=';')
    print(f"✔️ Sub-bacia {subbacia} ({nome_estacao}) salva com {df_base.shape[1]-3} modelos: {caminho_saida.name}")

In [ ]:
# ------------------------------------------------------------------------------
# Script: Consolidar Vazões Anuais com Média Multimodelo (MGB + CLIMBra)
# ------------------------------------------------------------------------------
# Este script percorre os arquivos diários organizados por sub-bacia, contendo
# 19 GCMs (2015–2100), e calcula a média anual por modelo.
#
# Em seguida, calcula a média multimodelo para cada ano e salva os resultados
# em um novo CSV por sub-bacia com o sufixo '_anual.csv'.
# ------------------------------------------------------------------------------

import pandas as pd
from pathlib import Path

# === Caminhos ===
PASTA_BASE         = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245")
PASTA_DIARIOS      = PASTA_BASE / "diarios"
PASTA_ANUAIS       = PASTA_BASE / "anuais"
PASTA_ANUAIS.mkdir(parents=True, exist_ok=True)

CAMINHO_MAPEAMENTO = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_mini.csv")
df_mapeamento = pd.read_csv(CAMINHO_MAPEAMENTO, sep=";")

# === Dicionário de código_mini → (estacao_obs, nome_estacao) ===
dict_mapeamento = {
    str(row["estacao_obs"]): row["nome_estacao"]
    for _, row in df_mapeamento.iterrows()
}

# === Processar cada arquivo diário ===
for caminho_csv in PASTA_DIARIOS.glob("*.csv"):
    nome_arquivo = caminho_csv.stem  # ex: '65295000'
    codigo_estacao = nome_arquivo  # Assume nome do arquivo como código da estação

    # Verifica se a estação está no mapeamento
    nome_estacao = dict_mapeamento.get(codigo_estacao, "Desconhecida")

    # Lê o CSV
    df = pd.read_csv(caminho_csv, sep=';')
    df.columns = [c.lower() for c in df.columns]

    # Cria coluna de data e extrai ano
    df['data'] = pd.to_datetime(dict(year=df['ano'], month=df['mes'], day=df['dia']), errors='coerce')
    df = df.dropna(subset=['data'])
    df['ano'] = df['data'].dt.year

    # Identifica colunas de modelos (exclui data)
    col_modelos = [col for col in df.columns if col not in ['dia', 'mes', 'data', 'ano']]

    # Média anual por modelo
    df_anual = df.groupby('ano')[col_modelos].mean().reset_index()

    # Média multimodelo
    df_anual['media_multimodelo'] = df_anual[col_modelos].mean(axis=1)

    # Salvar com mesmo nome, sufixo _anual.csv
    nome_saida = f"{codigo_estacao}_anual.csv"
    caminho_saida = PASTA_ANUAIS / nome_saida
    df_anual.to_csv(caminho_saida, index=False, sep=';')

    print(f"📘 Estação {codigo_estacao} ({nome_estacao}) — Série anual salva: {nome_saida}")

In [ ]:
# ------------------------------------------------------------------------------
# Script: Plotar Séries Anuais com Média Multimodelo e Intervalos de Incerteza
# ------------------------------------------------------------------------------
# Gráfico 1: Linhas de todos os GCMs (cinza) + Média Multimodelo (destacada) + Reta de Tendência (Sen)
# Gráfico 2: Média Multimodelo + Faixas de Incerteza (P10–P90 e Mín–Máx)
#              + Reta de Tendência com box de slope e p-valor do ensemble
# ------------------------------------------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import linregress
import pymannkendall as mk

# Caminhos
PASTA_DADOS = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245\anuais")
PASTA_SAIDA = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245") / "graficos_anuais_"
PASTA_SAIDA.mkdir(exist_ok=True)

# Loop pelas sub-bacias
for caminho_csv in PASTA_DADOS.glob("*_anual.csv"):
    subbacia = caminho_csv.stem.replace("_anual", "")
    df = pd.read_csv(caminho_csv, sep=';')

    col_modelos = [col for col in df.columns if col not in ['ano', 'media_multimodelo']]

    if not col_modelos:
        print(f"⚠️ Nenhum modelo encontrado em {caminho_csv.name}. Pulando...")
        continue

    # Cálculo da reta de tendência (ensemble)
    x = df['ano']
    y = df['media_multimodelo']
    res_mk = mk.hamed_rao_modification_test(y)
    reg = linregress(x, y)
    slope = reg.slope
    intercept = reg.intercept
    linha_tendencia = slope * x + intercept
    p_valor = res_mk.p

    # Texto da caixa informativa
    texto_box = f"Slope = {slope:.2f} m³/s/ano\np-valor = {p_valor:.4f}"

    # -------------------------------------------------------------------
    # GRÁFICO 1 — Todos os modelos (cinza) + Média multimodelo destacada + reta
    # -------------------------------------------------------------------
    plt.figure(figsize=(12, 6))
    for modelo in col_modelos:
        plt.plot(df['ano'], df[modelo], color='lightgray', linewidth=1)
    plt.plot(df['ano'], y, color='black', linewidth=2, label='Média Multimodelo')
    plt.plot(df['ano'], linha_tendencia, color='red', linestyle='--', label='Tendência')
    plt.title(f"Série Anual - Estação {subbacia}")
    plt.xlabel("Ano")
    plt.ylabel("Vazão média anual (m³/s)")
    plt.legend()
    plt.text(0.98, 0.95, texto_box, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle="round", facecolor='white', edgecolor='gray'))
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(PASTA_SAIDA / f"{subbacia}_todos_modelos_media.png", dpi=300)
    plt.close()

    # -------------------------------------------------------------------
    # GRÁFICO 2 — Média multimodelo + faixa P10–P90 e Min–Max + reta
    # -------------------------------------------------------------------
    df['p10'] = df[col_modelos].quantile(0.10, axis=1)
    df['p90'] = df[col_modelos].quantile(0.90, axis=1)
    df['min'] = df[col_modelos].min(axis=1)
    df['max'] = df[col_modelos].max(axis=1)

    plt.figure(figsize=(12, 6))
    plt.fill_between(df['ano'], df['min'], df['max'], color='gray', alpha=0.2, label="Intervalo Mín–Máx")
    plt.fill_between(df['ano'], df['p10'], df['p90'], color='blue', alpha=0.2, label="Intervalo P10–P90")
    plt.plot(df['ano'], y, color='black', linewidth=2, label='Média Multimodelo')
    plt.plot(df['ano'], linha_tendencia, color='red', linestyle='--', label='Tendência')
    plt.title(f"Incerteza Multimodelo - Estação {subbacia}")
    plt.xlabel("Ano")
    plt.ylabel("Vazão média anual (m³/s)")
    plt.legend()
    plt.text(0.98, 0.95, texto_box, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle="round", facecolor='white', edgecolor='gray'))
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(PASTA_SAIDA / f"{subbacia}_media_intervalos.png", dpi=300)
    plt.close()

    print(f"✔️ Gráficos salvos para estação {subbacia} em: {PASTA_SAIDA}")

In [ ]:
"""
Geração de séries (ensemble mean/median + P10–P90), estatísticas e tabelas
para todas as estações anuais (arquivos *_anual.csv) em SSP2-4.5 e SSP5-8.5.

ENTRADAS (pastas):
- E:\IGUAÇU_OTTO\8_Resultados_ssp245\anuais\*.csv
- E:\IGUAÇU_OTTO\8_Resultados_ssp585\anuais\*.csv

SAÍDAS:
1) Figuras (PNG) por estação (no estilo 2 painéis empilhados: SSP2-4.5 e SSP5-8.5),
   com linhas de média e mediana do ensemble e faixa de incerteza P10–P90:
   - OUT_DIR\Figuras\serie_ensemble_<estacao>.png

2) Excel consolidado (pronto para dissertação):
   - OUT_DIR\estatisticas_ensemble_todas_estacoes.xlsx
   com abas:
   - resumo_horizontes
   - estatisticas_series
   - lista_estacoes

3) CSVs consolidados:
   - OUT_DIR\resumo_horizontes_todas_estacoes.csv
   - OUT_DIR\estatisticas_series_todas_estacoes.csv
   - OUT_DIR\lista_estacoes_todas_estacoes.csv

OBSERVAÇÃO METODOLÓGICA:
- "ensemble_mean" é lido da coluna 'media_multimodelo' (média anual multimodelo).
- "ensemble_median" é a mediana entre os modelos (colunas dos modelos), ano a ano.
- P10 e P90 são percentis (10% e 90%) entre os modelos, ano a ano.
"""

from __future__ import annotations

from pathlib import Path
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------------------------------
# CONFIG
# -------------------------------------------------------
DIR_SSP245 = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245\anuais")
DIR_SSP585 = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp585 - Clima\8_Resultados_ssp585\anuais")

OUT_DIR = Path(r"E:\IGUAÇU_OTTO\9_Figuras_e_Tabelas_Ensemble_AB2")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_FIG_DIR = OUT_DIR / "Figuras"
OUT_FIG_DIR.mkdir(parents=True, exist_ok=True)

OUT_XLSX = OUT_DIR / "estatisticas_ensemble_todas_estacoes.xlsx"
OUT_CSV_RESUMO = OUT_DIR / "resumo_horizontes_todas_estacoes.csv"
OUT_CSV_STATS = OUT_DIR / "estatisticas_series_todas_estacoes.csv"
OUT_CSV_LISTA = OUT_DIR / "lista_estacoes_todas_estacoes.csv"

SEP = ";"

HORIZONTES = {
    "2015-2040": (2015, 2040),
    "2041-2070": (2041, 2070),
    "2071-2100": (2071, 2100),
    "2015-2100": (2015, 2100),
}

COL_ANO = "ano"
COL_MEAN = "media_multimodelo"  # já vem no arquivo

# -------------------------------------------------------
# FUNÇÕES
# -------------------------------------------------------
def extrair_codigo_estacao(path: Path) -> str:
    m = re.match(r"(\d+)_anual", path.stem)
    return m.group(1) if m else path.stem

def listar_arquivos_por_estacao(dir_csv: Path) -> dict[str, Path]:
    """Retorna dict {estacao: path_csv} para todos os *_anual.csv na pasta."""
    d: dict[str, Path] = {}
    for p in sorted(dir_csv.glob("*_anual.csv")):
        est = extrair_codigo_estacao(p)
        d[est] = p
    return d

def ler_csv_anual(path_csv: Path) -> tuple[pd.DataFrame, list[str]]:
    """
    Lê o CSV anual e retorna (df, cols_modelos).
    df contém:
      ano, ensemble_mean, ensemble_median, p10, p90
    """
    df = pd.read_csv(path_csv, sep=SEP)

    if COL_ANO not in df.columns:
        raise ValueError(f"{path_csv.name}: não encontrei coluna '{COL_ANO}'. Colunas: {list(df.columns)}")
    if COL_MEAN not in df.columns:
        raise ValueError(f"{path_csv.name}: não encontrei coluna '{COL_MEAN}'. Colunas: {list(df.columns)}")

    df[COL_ANO] = pd.to_numeric(df[COL_ANO], errors="coerce").astype("Int64")

    cols_modelos = [c for c in df.columns if c not in {COL_ANO, COL_MEAN}]
    if len(cols_modelos) == 0:
        raise ValueError(f"{path_csv.name}: não encontrei colunas de modelos (além de 'ano' e 'media_multimodelo').")

    df[cols_modelos] = df[cols_modelos].apply(pd.to_numeric, errors="coerce")

    df["ensemble_mean"] = pd.to_numeric(df[COL_MEAN], errors="coerce")
    df["ensemble_median"] = df[cols_modelos].median(axis=1, skipna=True)
    df["p10"] = df[cols_modelos].quantile(0.10, axis=1, interpolation="linear")
    df["p90"] = df[cols_modelos].quantile(0.90, axis=1, interpolation="linear")

    df = df.dropna(subset=[COL_ANO]).sort_values(COL_ANO).reset_index(drop=True)
    return df, cols_modelos

def recortar_periodo(df: pd.DataFrame, ano_ini: int, ano_fim: int) -> pd.DataFrame:
    return df[(df[COL_ANO] >= ano_ini) & (df[COL_ANO] <= ano_fim)].copy()

def estatisticas_series(df: pd.DataFrame) -> dict:
    """Estatísticas básicas das séries anuais (mean vs median do ensemble)."""
    x = df["ensemble_mean"].to_numpy(dtype=float)
    y = df["ensemble_median"].to_numpy(dtype=float)

    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]

    if len(x) == 0:
        return {
            "n": 0,
            "std_mean": np.nan,
            "std_median": np.nan,
            "var_mean": np.nan,
            "var_median": np.nan,
            "cov_mean_median": np.nan,
            "corr_mean_median": np.nan,
            "cv_mean": np.nan,
            "cv_median": np.nan,
        }

    std_mean = float(np.std(x, ddof=1)) if len(x) > 1 else 0.0
    std_median = float(np.std(y, ddof=1)) if len(y) > 1 else 0.0
    var_mean = float(np.var(x, ddof=1)) if len(x) > 1 else 0.0
    var_median = float(np.var(y, ddof=1)) if len(y) > 1 else 0.0

    if len(x) > 1:
        cov = float(np.cov(x, y, ddof=1)[0, 1])
        corr = float(np.corrcoef(x, y)[0, 1])
    else:
        cov, corr = 0.0, 1.0

    mean_x = float(np.mean(x))
    mean_y = float(np.mean(y))
    cv_mean = float(std_mean / mean_x) if np.isfinite(mean_x) and mean_x != 0 else np.nan
    cv_median = float(std_median / mean_y) if np.isfinite(mean_y) and mean_y != 0 else np.nan

    return {
        "n": int(len(x)),
        "std_mean": std_mean,
        "std_median": std_median,
        "var_mean": var_mean,
        "var_median": var_median,
        "cov_mean_median": cov,
        "corr_mean_median": corr,
        "cv_mean": cv_mean,
        "cv_median": cv_median,
    }

def resumo_horizontes(df: pd.DataFrame, horizontes: dict) -> list[dict]:
    """
    Métricas por horizonte:
    - mean e median do ensemble_mean
    - mean e median do ensemble_median
    - mean do P10 e do P90 no horizonte
    """
    rows: list[dict] = []
    for nome, (a0, a1) in horizontes.items():
        d = recortar_periodo(df, a0, a1)
        if len(d) == 0:
            rows.append({
                "horizonte": nome,
                "ano_ini": a0,
                "ano_fim": a1,
                "n_anos": 0,
                "mean_ensemble_mean": np.nan,
                "median_ensemble_mean": np.nan,
                "mean_ensemble_median": np.nan,
                "median_ensemble_median": np.nan,
                "mean_p10": np.nan,
                "mean_p90": np.nan,
            })
            continue

        rows.append({
            "horizonte": nome,
            "ano_ini": a0,
            "ano_fim": a1,
            "n_anos": int(len(d)),
            "mean_ensemble_mean": float(d["ensemble_mean"].mean()),
            "median_ensemble_mean": float(d["ensemble_mean"].median()),
            "mean_ensemble_median": float(d["ensemble_median"].mean()),
            "median_ensemble_median": float(d["ensemble_median"].median()),
            "mean_p10": float(d["p10"].mean()),
            "mean_p90": float(d["p90"].mean()),
        })
    return rows

def plotar_estacao_2paineis_com_faixa(df245: pd.DataFrame, df585: pd.DataFrame, estacao: str, out_png: Path) -> None:
    """
    Figura por estação, com 2 painéis empilhados (estilo do exemplo):
      - Painel 1: SSP2-4.5 (média, mediana e faixa P10–P90)
      - Painel 2: SSP5-8.5 (média, mediana e faixa P10–P90)
    """
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 8), dpi=300, sharex=True)

    # Painel SSP2-4.5
    ax = axes[0]
    ax.fill_between(df245[COL_ANO], df245["p10"], df245["p90"], alpha=0.20, label="P10–P90")
    ax.plot(df245[COL_ANO], df245["ensemble_mean"], linewidth=2, label="Ensemble – média anual")
    ax.plot(df245[COL_ANO], df245["ensemble_median"], linewidth=2, label="Ensemble – mediana anual")
    ax.set_title(f"SSP2-4.5 ({estacao})")
    ax.set_ylabel("Vazão (m³/s)")
    ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.6)
    ax.legend(loc="upper left", fontsize=9)

    # Painel SSP5-8.5
    ax = axes[1]
    ax.fill_between(df585[COL_ANO], df585["p10"], df585["p90"], alpha=0.20, label="P10–P90")
    ax.plot(df585[COL_ANO], df585["ensemble_mean"], linewidth=2, label="Ensemble – média anual")
    ax.plot(df585[COL_ANO], df585["ensemble_median"], linewidth=2, label="Ensemble – mediana anual")
    ax.set_title(f"SSP5-8.5 ({estacao})")
    ax.set_xlabel("Ano")
    ax.set_ylabel("Vazão (m³/s)")
    ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.6)
    ax.legend(loc="upper left", fontsize=9)

    fig.suptitle(
        "Série temporal do ensemble – comparação entre média e mediana (vazões anuais)",
        fontsize=14, fontweight="bold", y=0.98
    )
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    fig.savefig(out_png, dpi=300)
    plt.close(fig)

# -------------------------------------------------------
# MAIN
# -------------------------------------------------------
if __name__ == "__main__":

    arqs_245 = listar_arquivos_por_estacao(DIR_SSP245)
    arqs_585 = listar_arquivos_por_estacao(DIR_SSP585)

    estacoes = sorted(set(arqs_245.keys()) & set(arqs_585.keys()))
    if not estacoes:
        raise RuntimeError("Não encontrei estações em comum entre SSP2-4.5 e SSP5-8.5.")

    print(f"Estações em comum: {len(estacoes)}")

    resumo_rows: list[dict] = []
    stats_rows: list[dict] = []
    lista_rows: list[dict] = []

    for estacao in estacoes:
        p245 = arqs_245[estacao]
        p585 = arqs_585[estacao]

        df245, cols245 = ler_csv_anual(p245)
        df585, cols585 = ler_csv_anual(p585)

        # Figura no estilo 2 painéis (como seu exemplo)
        out_png = OUT_FIG_DIR / f"serie_ensemble_{estacao}.png"
        plotar_estacao_2paineis_com_faixa(df245, df585, estacao, out_png)

        # Estatísticas por cenário
        stats245 = estatisticas_series(df245)
        stats585 = estatisticas_series(df585)
        stats_rows.append({"cenario": "SSP2-4.5", "estacao": estacao, **stats245})
        stats_rows.append({"cenario": "SSP5-8.5", "estacao": estacao, **stats585})

        # Resumo por horizontes
        for row in resumo_horizontes(df245, HORIZONTES):
            resumo_rows.append({"cenario": "SSP2-4.5", "estacao": estacao, **row})
        for row in resumo_horizontes(df585, HORIZONTES):
            resumo_rows.append({"cenario": "SSP5-8.5", "estacao": estacao, **row})

        # Auditoria
        lista_rows.append({
            "estacao": estacao,
            "arquivo_ssp245": str(p245),
            "arquivo_ssp585": str(p585),
            "n_modelos_cols_ssp245": int(len(cols245)),
            "n_modelos_cols_ssp585": int(len(cols585)),
            "ano_min_ssp245": int(df245[COL_ANO].min()),
            "ano_max_ssp245": int(df245[COL_ANO].max()),
            "ano_min_ssp585": int(df585[COL_ANO].min()),
            "ano_max_ssp585": int(df585[COL_ANO].max()),
        })

    # Consolida DataFrames finais
    df_resumo = pd.DataFrame(resumo_rows)
    df_stats = pd.DataFrame(stats_rows)
    df_lista = pd.DataFrame(lista_rows)

    ordem_h = pd.CategoricalDtype(["2015-2040", "2041-2070", "2071-2100", "2015-2100"], ordered=True)
    df_resumo["horizonte"] = df_resumo["horizonte"].astype(ordem_h)
    df_resumo = df_resumo.sort_values(["cenario", "estacao", "horizonte"]).reset_index(drop=True)

    df_stats = df_stats.sort_values(["cenario", "estacao"]).reset_index(drop=True)
    df_lista = df_lista.sort_values(["estacao"]).reset_index(drop=True)

    # Exporta CSVs
    df_resumo.to_csv(OUT_CSV_RESUMO, index=False, encoding="utf-8-sig")
    df_stats.to_csv(OUT_CSV_STATS, index=False, encoding="utf-8-sig")
    df_lista.to_csv(OUT_CSV_LISTA, index=False, encoding="utf-8-sig")

    # Exporta Excel (3 abas)
    with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as writer:
        df_resumo.to_excel(writer, sheet_name="resumo_horizontes", index=False)
        df_stats.to_excel(writer, sheet_name="estatisticas_series", index=False)
        df_lista.to_excel(writer, sheet_name="lista_estacoes", index=False)

    print("\n✅ Concluído com sucesso!")
    print("Figuras (PNG):", OUT_FIG_DIR)
    print("Excel final:", OUT_XLSX)
    print("CSV resumo:", OUT_CSV_RESUMO)
    print("CSV estatísticas:", OUT_CSV_STATS)
    print("CSV lista:", OUT_CSV_LISTA)

In [ ]:
# ------------------------------------------------------------------------------
# Script: Consolidar Vazões Anuais com Média Multimodelo (MGB + CLIMBra)
# ------------------------------------------------------------------------------
# Agora, além da vazão média anual, calcula também:
#  - vazão máxima anual
#  - vazão mínima anual
#  - percentil 10% anual
#  - percentil 90% anual
#
# Para cada modelo, ano a ano, a partir da série diária.
# Em seguida, calcula a média multimodelo das MÉDIAS anuais.
# ------------------------------------------------------------------------------

import pandas as pd
from pathlib import Path

# === Caminhos ===
PASTA_BASE         = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245")
PASTA_DIARIOS      = PASTA_BASE / "diarios"
PASTA_ANUAIS       = PASTA_BASE / "anuais_est"
PASTA_ANUAIS.mkdir(parents=True, exist_ok=True)

CAMINHO_MAPEAMENTO = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Estações_mini.csv")
df_mapeamento = pd.read_csv(CAMINHO_MAPEAMENTO, sep=";")

# === Dicionário de código_mini → (estacao_obs, nome_estacao) ===
dict_mapeamento = {
    str(row["estacao_obs"]): row["nome_estacao"]
    for _, row in df_mapeamento.iterrows()
}

# Funções auxiliares para os quantis
def q10(x):
    return x.quantile(0.10)

def q90(x):
    return x.quantile(0.90)

# === Processar cada arquivo diário ===
for caminho_csv in PASTA_DIARIOS.glob("*.csv"):
    nome_arquivo = caminho_csv.stem  # ex: '65295000'
    codigo_estacao = nome_arquivo    # Assume nome do arquivo como código da estação

    # Verifica se a estação está no mapeamento
    nome_estacao = dict_mapeamento.get(codigo_estacao, "Desconhecida")

    # Lê o CSV
    df = pd.read_csv(caminho_csv, sep=';')
    df.columns = [c.lower() for c in df.columns]

    # Cria coluna de data e extrai ano
    df['data'] = pd.to_datetime(
        dict(year=df['ano'], month=df['mes'], day=df['dia']),
        errors='coerce'
    )
    df = df.dropna(subset=['data'])
    df['ano'] = df['data'].dt.year

    # Identifica colunas de modelos (exclui data/ano/dia/mes)
    col_modelos = [col for col in df.columns if col not in ['dia', 'mes', 'data', 'ano']]

    # Estatísticas anuais por modelo: média, máx, mín, p10 e p90
    df_anual = df.groupby('ano')[col_modelos].agg(['mean', 'max', 'min', q10, q90])

    # Flatten do MultiIndex de colunas: modelo_estatistica
    df_anual.columns = [
        f"{modelo}_{estat}" for (modelo, estat) in df_anual.columns.to_list()
    ]
    df_anual = df_anual.reset_index()

    # Média multimodelo das MÉDIAS anuais (mantendo sua lógica original)
    cols_medias = [c for c in df_anual.columns if c.endswith('_mean')]
    df_anual['media_multimodelo'] = df_anual[cols_medias].mean(axis=1)

    # Salvar com mesmo nome, sufixo _anual.csv
    nome_saida = f"{codigo_estacao}_anual.csv"
    caminho_saida = PASTA_ANUAIS / nome_saida
    df_anual.to_csv(caminho_saida, index=False, sep=';')

    print(f"📘 Estação {codigo_estacao} ({nome_estacao}) — Série anual salva: {nome_saida}")

In [ ]:
# ------------------------------------------------------------------------------
# Script: Plotar Séries Anuais para Várias Estatísticas (MGB + CLIMBra)
# ------------------------------------------------------------------------------
# Para cada sub-bacia e para cada estatística (mean, max, min, q10, q90):
#
#   Gráfico 1: Linhas de todos os GCMs (cinza) + Média Multimodelo (destacada) + Reta de Tendência
#   Gráfico 2: Média Multimodelo + Faixas de Incerteza (P10–P90 e Mín–Máx) + Reta de Tendência
#
# Os CSVs anuais têm o formato:
#   ano,
#   MODELO_mean, MODELO_max, MODELO_min, MODELO_q10, MODELO_q90,
#   media_multimodelo (para a média; opcionalmente recalculada)
# ------------------------------------------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import linregress
import pymannkendall as mk

# Caminhos
PASTA_DADOS = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245\anuais_est")
PASTA_SAIDA = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245") / "graficos_anuais_est"
PASTA_SAIDA.mkdir(exist_ok=True)

# Dicionário para rótulos bonitos no eixo Y e no título
ROTULO_METRICA = {
    "mean": "Vazão média anual (m³/s)",
    "max":  "Vazão máxima anual (m³/s)",
    "min":  "Vazão mínima anual (m³/s)",
    "q10":  "Vazão anual - P10 (m³/s)",
    "q90":  "Vazão anual - P90 (m³/s)",
}

TITULO_METRICA = {
    "mean": "média",
    "max":  "máxima",
    "min":  "mínima",
    "q10":  "P10",
    "q90":  "P90",
}

# Métricas disponíveis nos arquivos
METRICAS = ["mean", "max", "min", "q10", "q90"]

# Loop pelas sub-bacias
for caminho_csv in PASTA_DADOS.glob("*_anual.csv"):
    subbacia = caminho_csv.stem.replace("_anual", "")
    df = pd.read_csv(caminho_csv, sep=';')

    if 'ano' not in df.columns:
        print(f"⚠️ Coluna 'ano' não encontrada em {caminho_csv.name}. Pulando...")
        continue

    anos = df['ano']

    # Para cada métrica (mean, max, min, q10, q90), gerar dois gráficos
    for metrica in METRICAS:
        # Seleciona apenas as colunas dessa métrica: *_mean, *_max, etc.
        col_metricas = [c for c in df.columns if c.endswith(f"_{metrica}")]

        if len(col_metricas) == 0:
            print(f"⚠️ Nenhuma coluna com a métrica '{metrica}' em {caminho_csv.name}. Pulando métrica...")
            continue

        # --- Série ensemble (média multimodelo daquela métrica) ---
        # Se for a média e já existir 'media_multimodelo', aproveita; senão, recalcula
        if metrica == "mean" and "media_multimodelo" in df.columns:
            y_ensemble = df["media_multimodelo"].copy()
        else:
            y_ensemble = df[col_metricas].mean(axis=1)

        # Teste de tendência (Mann-Kendall + regressão linear) para essa métrica
        res_mk = mk.hamed_rao_modification_test(y_ensemble)
        reg = linregress(anos, y_ensemble)
        slope = reg.slope
        intercept = reg.intercept
        linha_tendencia = slope * anos + intercept
        p_valor = res_mk.p

        texto_box = (
            f"Métrica: {TITULO_METRICA.get(metrica, metrica)}\n"
            f"Slope = {slope:.2f} m³/s/ano\n"
            f"p-valor = {p_valor:.4f}"
        )

       #  # -------------------------------------------------------------------
       #  # GRÁFICO 1 — Todos os modelos (cinza) + Média multimodelo (metric) + reta
       #  # -------------------------------------------------------------------
       # plt.figure(figsize=(12, 6))
        for col in col_metricas:
            plt.plot(anos, df[col], color='lightgray', linewidth=1)

        plt.plot(anos, y_ensemble, color='black', linewidth=2,
                 label=f"Média Multimodelo ({TITULO_METRICA.get(metrica, metrica)})")
        plt.plot(anos, linha_tendencia, color='red', linestyle='--', label='Tendência')

        plt.title(f"Série Anual {TITULO_METRICA.get(metrica, metrica)} - Estação {subbacia}")
        plt.xlabel("Ano")
        plt.ylabel(ROTULO_METRICA.get(metrica, "Vazão (m³/s)"))
        plt.legend()
        plt.text(
            0.98, 0.95, texto_box, transform=plt.gca().transAxes,
            fontsize=10, verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle="round", facecolor='white', edgecolor='gray')
        )
        plt.grid(True)
        plt.tight_layout()

        nome_fig1 = f"{subbacia}_{metrica}_todos_modelos_media.png"
        plt.savefig(PASTA_SAIDA / nome_fig1, dpi=300)
        plt.close()

        # -------------------------------------------------------------------
        # GRÁFICO 2 — Ensemble + faixa P10–P90 e Min–Max entre modelos (para a métrica)
        # -------------------------------------------------------------------
        # Aqui, os intervalos são para a métrica selecionada (por ex., max entre modelos,
        # P10 entre modelos, etc.).
        p10 = df[col_metricas].quantile(0.10, axis=1)
        p90 = df[col_metricas].quantile(0.90, axis=1)
        vmin = df[col_metricas].min(axis=1)
        vmax = df[col_metricas].max(axis=1)

        plt.figure(figsize=(12, 6))
        plt.fill_between(anos, vmin, vmax, alpha=0.2, label="Intervalo Mín–Máx")
        plt.fill_between(anos, p10, p90, alpha=0.2, label="Intervalo P10–P90")
        plt.plot(anos, y_ensemble, color='black', linewidth=2,
                 label=f"Média Multimodelo ({TITULO_METRICA.get(metrica, metrica)})")
        plt.plot(anos, linha_tendencia, color='red', linestyle='--', label='Tendência')

        plt.title(f"Incerteza Multimodelo ({TITULO_METRICA.get(metrica, metrica)}) - Estação {subbacia}")
        plt.xlabel("Ano")
        plt.ylabel(ROTULO_METRICA.get(metrica, "Vazão (m³/s)"))
        plt.legend()
        plt.text(
            0.98, 0.95, texto_box, transform=plt.gca().transAxes,
            fontsize=10, verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle="round", facecolor='white', edgecolor='gray')
        )
        plt.grid(True)
        plt.tight_layout()

        nome_fig2 = f"{subbacia}_{metrica}_media_intervalos.png"
        plt.savefig(PASTA_SAIDA / nome_fig2, dpi=300)
        plt.close()

        print(f"✔️ Gráficos ({metrica}) salvos para estação {subbacia} em: {PASTA_SAIDA}")

In [ ]:
from pathlib import Path
import pandas as pd

# Caminho da pasta de entrada
PASTA_DADOS = Path(r"C:\Users\Matheus Marinho\Desktop\IGUAÇU_OTTO\3_Estações FLU\Input")

# Lista para armazenar as médias por estação
medias = []

# Itera sobre os arquivos CSV
for caminho_csv in sorted(PASTA_DADOS.glob("*.csv")):
    try:
        df = pd.read_csv(
            caminho_csv,
            sep=';',
            header=None,
            names=["dia", "mes", "ano", "valor"],
            dtype=str
        )

        # Converte valor diretamente, pois já está com ponto como separador decimal
        df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

        # Valores -1 são considerados como nulos
        df.loc[df["valor"] == -1, "valor"] = pd.NA

        # Filtra apenas anos entre 1980 e 2010
        df["ano"] = pd.to_numeric(df["ano"], errors='coerce')
        df_base = df[(df["ano"] >= 1980) & (df["ano"] <= 2023)]

        media = df_base["valor"].mean(skipna=True)

        medias.append({
            "estacao_obs": caminho_csv.stem,
            "media_base": round(media, 2)
        })

    except Exception as e:
        print(f"⚠️ Erro ao processar {caminho_csv.name}: {e}")

# Exibir resultado
df_resultado = pd.DataFrame(medias)
print(df_resultado)

# Salvar como CSV (separador ponto e vírgula)
df_resultado.to_csv(PASTA_DADOS / "media_periodo_base.csv", sep=';', index=False, encoding='utf-8-sig')


In [ ]:
# ------------------------------------------------------------------------------
# Script: Analise de Tendência Hidrológica Multimodelo - MGB + CLIMBra (CMIP6)
# Autor: Matheus Marinho
# Objetivo:
#   - Avaliar tendências de vazões anuais simuladas pelo modelo MGB,
#     com base em 19 modelos climáticos CMIP6 (cenário SSP5-8.5) do conjunto CLIMBra.
#
# Metodologia:
#   - Aplicação do Teste de Mann-Kendall Modificado (Hamed e Rao, 1998)
#   - Estimativa da inclinação de Theil-Sen para magnitude da tendência
#   - Avaliação por horizonte temporal: Curto (2015–2040), Médio (2041–2070),
#     Longo (2071–2100) e Total (2015–2100)
#   - Classificação de concordância entre modelos com base em:
#       • p-valor < 0.05 (significância estatística)
#       • variação relativa ≥ 10% em relação à média histórica (1980–2023)
#
# Saídas:
#   - resumo_tendencias_multimodelo.csv:
#       Tabela síntese por sub-bacia e período com métricas agregadas e estatísticas do ensemble
#   - Detalhado_por_modelo/{sub_bacia}_{periodo}.csv:
#       Tendência individual por modelo: slope, p-valor, variação relativa, significância
#
# Requisitos:
#   - Python 3.x
#   - Pacotes: pandas, numpy, pymannkendall, scipy
#
# Data de execução: Junho de 2025
# ------------------------------------------------------------------------------


import pandas as pd
import numpy as np
import pymannkendall as mk
from pathlib import Path
from scipy.stats import linregress

# === CONFIGURAÇÕES ===
PASTA_ANUAIS = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245/anuais")
PASTA_SAIDA = PASTA_ANUAIS.parent / "Concordancia"
PASTA_DETALHADO = PASTA_SAIDA / "Detalhado_por_modelo"
PASTA_SAIDA.mkdir(exist_ok=True)
PASTA_DETALHADO.mkdir(exist_ok=True)

# Leitura da média histórica (1980–2023)
media_base = pd.read_csv(
    r"E:/IGUAÇU_OTTO/3_Estações FLU/Input/media_periodo_base.csv",
    sep=';', dtype={"estacao_obs": str}
)
media_base["estacao_obs"] = media_base["estacao_obs"].str.strip()

# Períodos de análise
PERIODOS = {
    "Curto": (2015, 2040),
    "Médio": (2041, 2070),
    "Longo": (2071, 2100),
    "Total": (2015, 2100),
}

# Loop principal por sub-bacia
tabela_final = []

for caminho in PASTA_ANUAIS.glob("*.csv"):
    nome_sb = caminho.stem.replace("_anual", "").strip()

    df = pd.read_csv(caminho, sep=";")
    if "ano" not in df.columns:
        print(f"⚠️ Arquivo {caminho.name} não possui coluna 'ano'. Pulando...")
        continue
    df = df.set_index("ano")

    # Identifica colunas de modelos (exclui 'media_multimodelo')
    col_modelos = [col for col in df.columns if col != "media_multimodelo"]

    linha = media_base.loc[media_base["estacao_obs"] == nome_sb]
    if linha.empty:
        print(f"⚠️ Média histórica não encontrada para sub-bacia {nome_sb}. Pulando...")
        continue
    media_hist = linha["media_base"].values[0]

    for nome_periodo, (ini, fim) in PERIODOS.items():
        df_per = df.loc[ini:fim, col_modelos]
        resultados_modelos = []

        total = 0
        positivos = 0
        negativos = 0
        significativos = 0

        for modelo in df_per.columns:
            serie = df_per[modelo].dropna()
            if len(serie) < 10:
                continue

            try:
                resultado = mk.hamed_rao_modification_test(serie)
                slope = linregress(serie.index, serie.values).slope
                var_relativa = (slope * len(serie)) / media_hist
                significancia = resultado.p < 0.05 and abs(var_relativa) >= 0.10

                if significancia:
                    significativos += 1
                    if slope > 0:
                        positivos += 1
                    else:
                        negativos += 1

                total += 1

                resultados_modelos.append({
                    "modelo": modelo,
                    "slope": round(slope, 4),
                    "p_valor": round(resultado.p, 4),
                    "var_relativa_%": round(var_relativa * 100, 2),
                    "significativo": significancia,
                    "sinal": "positivo" if slope > 0 else "negativo"
                })

            except Exception as e:
                print(f"[ERRO] {modelo} - {nome_sb} ({nome_periodo}): {e}")

        # Salvar resultados individuais
        df_result_ind = pd.DataFrame(resultados_modelos)
        df_result_ind.to_csv(PASTA_DETALHADO / f"{nome_sb}_{nome_periodo}.csv", sep=';', index=False, encoding='utf-8-sig')

        # Ensemble multimodelo (coluna media_multimodelo)
        df_ens = df.loc[ini:fim, "media_multimodelo"].dropna()
        ens_result = mk.hamed_rao_modification_test(df_ens)
        ens_slope = linregress(df_ens.index, df_ens.values).slope
        ens_intercept = linregress(df_ens.index, df_ens.values).intercept
        ens_var_relativa = (ens_slope * len(df_ens)) / media_hist
        ens_significativo = ens_result.p < 0.05 and abs(ens_var_relativa) >= 0.10
        ens_sinal = "positivo" if ens_slope > 0 else "negativo"

        # Classificação da concordância
        prop = significativos / total if total > 0 else 0
        if prop >= 0.66:
            classe = "robusta"
        elif prop >= 0.5:
            classe = "moderada"
        else:
            classe = "sem consenso"

        sinal = "aumento" if positivos > negativos else ("redução" if negativos > positivos else "neutro")

        tabela_final.append({
            "sub_bacia": nome_sb,
            "periodo": nome_periodo,
            "modelos_total": total,
            "significativos": significativos,
            "positivos": positivos,
            "negativos": negativos,
            "concordancia_%": round(prop * 100, 1),
            "classe_sinal": f"{sinal} {classe}",
            "ensemble_trend": ens_result.trend,
            "ensemble_p": round(ens_result.p, 4),
            "ensemble_slope": round(ens_slope, 4),
            "ensemble_intercept": round(ens_intercept, 2),
            "ensemble_var_relativa_%": round(ens_var_relativa * 100, 2),
            "ensemble_significativo": ens_significativo,
            "ensemble_sinal": ens_sinal
        })

# Exporta resumo final
pd.DataFrame(tabela_final).to_csv(
    PASTA_SAIDA / "resumo_tendencias_multimodelo.csv",
    sep=';', index=False, encoding='utf-8-sig'
)

print("\n✔️ Análise completa salva com sucesso!")

In [ ]:
# ------------------------------------------------------------------------------
# Script: Analise de Tendência Hidrológica Multimodelo - MGB + CLIMBra (CMIP6)
# Autor: Matheus Marinho
# Objetivo:
#   - Avaliar tendências de vazões anuais simuladas pelo modelo MGB,
#     com base em 19 modelos climáticos CMIP6 (cenário SSP5-8.5) do conjunto CLIMBra.
#
# Agora:
#   - A análise é feita separadamente para as estatísticas anuais:
#       • vazão mínima (min)
#       • vazão média (mean)
#       • vazão máxima (max)
#       • P10 (q10)
#       • P90 (q90)
#
# Metodologia:
#   - Aplicação do Teste de Mann-Kendall Modificado (Hamed e Rao, 1998)
#   - Estimativa da inclinação de Theil-Sen / regressão linear (slope)
#   - Avaliação por horizonte temporal: Curto (2015–2040), Médio (2041–2070),
#     Longo (2071–2100) e Total (2015–2100)
#   - Classificação de concordância entre modelos com base em:
#       • p-valor < 0.05 (significância estatística)
#       • variação relativa ≥ 10% em relação à média histórica (1980–2023)
#
# Saídas:
#   - resumo_tendencias_multimodelo_metricas.csv:
#       Tabela síntese por sub-bacia, período e variável (min, mean, max, q10, q90)
#   - Detalhado_por_modelo/{sub_bacia}_{variavel}_{periodo}.csv:
#       Tendência individual por modelo: slope, p-valor, variação relativa, significância
# ------------------------------------------------------------------------------

import pandas as pd
import numpy as np
import pymannkendall as mk
from pathlib import Path
from scipy.stats import linregress

# === CONFIGURAÇÕES ===
PASTA_ANUAIS = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245\anuais_est")
PASTA_SAIDA = PASTA_ANUAIS.parent / "Concordancia_est"
PASTA_DETALHADO = PASTA_SAIDA / "Detalhado_por_modelo"
PASTA_SAIDA.mkdir(exist_ok=True)
PASTA_DETALHADO.mkdir(exist_ok=True)

# Leitura da média histórica (1980–2023)
media_base = pd.read_csv(
    r"E:/IGUAÇU_OTTO/3_Estações FLU/Input/media_periodo_base.csv",
    sep=';', dtype={"estacao_obs": str}
)
media_base["estacao_obs"] = media_base["estacao_obs"].str.strip()

# Períodos de análise
PERIODOS = {
    "Curto": (2015, 2040),
    "Médio": (2041, 2070),
    "Longo": (2071, 2100),
    "Total": (2015, 2100),
}

# Variáveis (estatísticas anuais) a analisar
VARIAVEIS = ["min", "mean", "max", "q10", "q90"]
NOME_VARIAVEL = {
    "min": "mínima",
    "mean": "média",
    "max": "máxima",
    "q10": "P10",
    "q90": "P90",
}

# Tabela resumo final (todas as sub-bacias, períodos e variáveis)
tabela_final = []

# Loop principal por sub-bacia
for caminho in PASTA_ANUAIS.glob("*.csv"):
    nome_sb = caminho.stem.replace("_anual", "").strip()

    df = pd.read_csv(caminho, sep=";")
    if "ano" not in df.columns:
        print(f"⚠️ Arquivo {caminho.name} não possui coluna 'ano'. Pulando...")
        continue

    df = df.set_index("ano")

    # Média histórica da sub-bacia
    linha = media_base.loc[media_base["estacao_obs"] == nome_sb]
    if linha.empty:
        print(f"⚠️ Média histórica não encontrada para sub-bacia {nome_sb}. Pulando...")
        continue
    media_hist = linha["media_base"].values[0]

    # Loop por variável (min, mean, max, q10, q90)
    for var in VARIAVEIS:
        # Colunas dos modelos correspondentes a essa variável (ex: modelo1_mean, modelo2_mean, ...)
        col_modelos_var = [c for c in df.columns if c.endswith(f"_{var}")]
        if len(col_modelos_var) == 0:
            print(f"⚠️ Nenhuma coluna encontrada para variável '{var}' em {caminho.name}. Pulando variável...")
            continue

        # Loop por período (Curto, Médio, Longo, Total)
        for nome_periodo, (ini, fim) in PERIODOS.items():
            df_per = df.loc[ini:fim, col_modelos_var]

            resultados_modelos = []

            total = 0
            positivos = 0
            negativos = 0
            significativos = 0

            # --- Análise modelo a modelo ---
            for modelo in df_per.columns:
                serie = df_per[modelo].dropna()
                if len(serie) < 10:
                    continue

                try:
                    # Teste de Mann-Kendall modificado
                    resultado = mk.hamed_rao_modification_test(serie)

                    # Regressão linear (slope em unidades de vazão / ano)
                    reg = linregress(serie.index.values, serie.values)
                    slope = reg.slope

                    # Variação relativa ao longo do período, em relação à média histórica
                    var_relativa = (slope * len(serie)) / media_hist  # fração da média histórica
                    significancia = (resultado.p < 0.05) and (abs(var_relativa) >= 0.10)

                    if significancia:
                        significativos += 1
                        if slope > 0:
                            positivos += 1
                        else:
                            negativos += 1

                    total += 1

                    resultados_modelos.append({
                        "sub_bacia": nome_sb,
                        "periodo": nome_periodo,
                        "variavel": NOME_VARIAVEL[var],
                        "modelo": modelo,
                        "slope": round(slope, 4),
                        "p_valor": round(resultado.p, 4),
                        "tau": round(resultado.Tau, 4),
                        "var_relativa_%": round(var_relativa * 100, 2),
                        "significativo": significancia,
                        "sinal": "positivo" if slope > 0 else "negativo"
                    })

                except Exception as e:
                    print(f"[ERRO] {modelo} - {nome_sb} ({nome_periodo}, {var}): {e}")

            # Salvar resultados individuais (se houver modelos analisados)
            df_result_ind = pd.DataFrame(resultados_modelos)
            if not df_result_ind.empty:
                nome_detalhe = f"{nome_sb}_{var}_{nome_periodo}.csv"
                df_result_ind.to_csv(
                    PASTA_DETALHADO / nome_detalhe,
                    sep=';', index=False, encoding='utf-8-sig'
                )

            # --- Ensemble multimodelo para a variável ---
            df_ens = df_per.mean(axis=1).dropna()
            if len(df_ens) >= 10:
                ens_result = mk.hamed_rao_modification_test(df_ens)
                reg_ens = linregress(df_ens.index.values, df_ens.values)
                ens_slope = reg_ens.slope
                ens_intercept = reg_ens.intercept
                ens_var_relativa = (ens_slope * len(df_ens)) / media_hist
                ens_significativo = (ens_result.p < 0.05) and (abs(ens_var_relativa) >= 0.10)
                ens_sinal = "positivo" if ens_slope > 0 else "negativo"
            else:
                # Caso extremo de poucos anos
                ens_result = None
                ens_slope = np.nan
                ens_intercept = np.nan
                ens_var_relativa = np.nan
                ens_significativo = False
                ens_sinal = "indefinido"

            # Classificação da concordância entre modelos
            prop = significativos / total if total > 0 else 0
            if prop >= 0.66:
                classe = "robusta"
            elif prop >= 0.5:
                classe = "moderada"
            else:
                classe = "sem consenso"

            if positivos > negativos:
                sinal_conjunto = "aumento"
            elif negativos > positivos:
                sinal_conjunto = "redução"
            else:
                sinal_conjunto = "neutro"

            tabela_final.append({
                "sub_bacia": nome_sb,
                "periodo": nome_periodo,
                "variavel": NOME_VARIAVEL[var],
                "modelos_total": total,
                "significativos": significativos,
                "positivos": positivos,
                "negativos": negativos,
                "concordancia_%": round(prop * 100, 1),
                "classe_sinal": f"{sinal_conjunto} {classe}",
                "ensemble_trend": ens_result.trend if ens_result is not None else "NA",
                "ensemble_p": round(ens_result.p, 4) if ens_result is not None else np.nan,
                "ensemble_tau": round(ens_result.Tau, 4) if ens_result is not None else np.nan,
                "ensemble_slope": round(ens_slope, 4) if not np.isnan(ens_slope) else np.nan,
                "ensemble_intercept": round(ens_intercept, 2) if not np.isnan(ens_intercept) else np.nan,
                "ensemble_var_relativa_%": round(ens_var_relativa * 100, 2) if not np.isnan(ens_var_relativa) else np.nan,
                "ensemble_significativo": ens_significativo,
                "ensemble_sinal": ens_sinal
            })

# Exporta resumo final (todas as variáveis e períodos)
pd.DataFrame(tabela_final).to_csv(
    PASTA_SAIDA / "resumo_tendencias_multimodelo_metricas.csv",
    sep=';', index=False, encoding='utf-8-sig'
)

print("\n✔️ Análise completa (min, média, max, P10, P90) salva com sucesso!")

In [ ]:
# ------------------------------------------------------------------------------
# Script: Comparação entre Cenários SSP2-4.5 e SSP5-8.5 com Incerteza Multimodelo
# ------------------------------------------------------------------------------
# Este script gera gráficos comparando os dois cenários de emissão para cada
# sub-bacia, com as seguintes camadas:
#   - Média multimodelo de cada cenário
#   - Faixas de incerteza (P10–P90) para cada cenário
#   - Retas de tendência (Sen) para as médias multimodelo
# ------------------------------------------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import linregress
import pymannkendall as mk

# Pastas de entrada
PASTA_SSP245 = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245/anuais")
PASTA_SSP585 = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp585 - Clima\8_Resultados_ssp585/anuais")
PASTA_SAIDA = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp585 - Clima\8_Resultados_ssp585/Comparacao_Cenarios")
PASTA_SAIDA.mkdir(exist_ok=True)

# Lista de sub-bacias baseada nos arquivos SSP2-4.5
for caminho_245 in PASTA_SSP245.glob("*_anual.csv"):
    nome_sb = caminho_245.stem.replace("_anual", "")
    caminho_585 = PASTA_SSP585 / f"{nome_sb}_anual.csv"

    if not caminho_585.exists():
        print(f"⚠️ Arquivo ausente para SSP5-8.5: {nome_sb}")
        continue

    # Leitura dos dois cenários
    df_245 = pd.read_csv(caminho_245, sep=';')
    df_585 = pd.read_csv(caminho_585, sep=';')

    # Validação
    if 'ano' not in df_245.columns or 'media_multimodelo' not in df_245.columns:
        print(f"⚠️ Dados incompletos em SSP2-4.5: {nome_sb}")
        continue
    if 'ano' not in df_585.columns or 'media_multimodelo' not in df_585.columns:
        print(f"⚠️ Dados incompletos em SSP5-8.5: {nome_sb}")
        continue

    anos = df_245['ano']

    # Incerteza SSP2-4.5
    col_mod_245 = [col for col in df_245.columns if col not in ['ano', 'media_multimodelo']]
    df_245['p10'] = df_245[col_mod_245].quantile(0.10, axis=1)
    df_245['p90'] = df_245[col_mod_245].quantile(0.90, axis=1)

    # Incerteza SSP5-8.5
    col_mod_585 = [col for col in df_585.columns if col not in ['ano', 'media_multimodelo']]
    df_585['p10'] = df_585[col_mod_585].quantile(0.10, axis=1)
    df_585['p90'] = df_585[col_mod_585].quantile(0.90, axis=1)

    # Tendência SSP2-4.5
    x = anos
    y_245 = df_245['media_multimodelo']
    mk_245 = mk.hamed_rao_modification_test(y_245)
    reg_245 = linregress(x, y_245)
    linha_245 = reg_245.slope * x + reg_245.intercept

    # Tendência SSP5-8.5
    y_585 = df_585['media_multimodelo']
    mk_585 = mk.hamed_rao_modification_test(y_585)
    reg_585 = linregress(x, y_585)
    linha_585 = reg_585.slope * x + reg_585.intercept

    # Plotagem
    plt.figure(figsize=(12, 6))
    plt.fill_between(x, df_245['p10'], df_245['p90'], color='blue', alpha=0.2, label='SSP2-4.5: P10–P90')
    plt.fill_between(x, df_585['p10'], df_585['p90'], color='red', alpha=0.2, label='SSP5-8.5: P10–P90')
    plt.plot(x, y_245, color='blue', linewidth=2, label='Média SSP2-4.5')
    plt.plot(x, y_585, color='red', linewidth=2, label='Média SSP5-8.5')
    plt.plot(x, linha_245, color='blue', linestyle='--', label='Tendência SSP2-4.5')
    plt.plot(x, linha_585, color='red', linestyle='--', label='Tendência SSP5-8.5')

    # Título e legenda
    plt.title(f"Comparação de Cenários - Estação {nome_sb}")
    plt.xlabel("Ano")
    plt.ylabel("Vazão média anual (m³/s)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Salva
    plt.savefig(PASTA_SAIDA / f"{nome_sb}_comparacao_cenarios.png", dpi=300)
    plt.close()

    print(f"✔️ Gráfico salvo para estação {nome_sb}")

In [ ]:
# ------------------------------------------------------------------------------
# Avaliação dos Resultados do MGB com Modelos CLIMBra/CMIP6
#
# Para cada estação gera:
#   1) Distribuições das vazões médias mensais (cada modelo x observado)
#      -> 1 figura POR MODELO, estilo violin + boxplot (como exemplo enviado)
#   2) Distribuições das vazões médias mensais (média dos modelos x observado)
#   3) Curvas de permanência diária (FDC) para cada modelo x observado
#   4) FDC para cada modelo x observado desconsiderando p1 e p99
#   5) FDC para a média dos modelos x observado (completa e sem extremos)
# ------------------------------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import math

# ================= CONFIGURAÇÕES DE PASTA =====================================

PASTA_DIARIOS = Path(r"E:\IGUAÇU_OTTO\7_Projeções_ssp245_clima\8_Resultados_ssp245\diarios")
PASTA_OBS     = Path(r"E:\IGUAÇU_OTTO\3_Estações FLU\Input\Observados")

PASTA_SAIDA   = PASTA_DIARIOS.parent / "avaliacao_modelos"
PASTA_SAIDA.mkdir(parents=True, exist_ok=True)

MES_LABELS = ["Jan", "Fev", "Mar", "Abr", "Mai", "Jun",
              "Jul", "Ago", "Set", "Out", "Nov", "Dez"]

# ==================== FUNÇÕES AUXILIARES ======================================

def carregar_dados(estacao: str):
    """
    Lê os dados diários de modelos e observados para uma estação.
    - modelos: PASTA_DIARIOS / '{estacao}.csv'  (com cabeçalho)
    - observado: PASTA_OBS / '{estacao}.csv'    (SEM cabeçalho, 4 colunas: dia;mes;ano;Qobs)
    """
    # --- Simulado (modelos) ---
    arq_mod = PASTA_DIARIOS / f"{estacao}.csv"
    if not arq_mod.exists():
        raise FileNotFoundError(f"Arquivo de modelos não encontrado: {arq_mod}")

    df_mod = pd.read_csv(arq_mod, sep=';')
    df_mod.columns = [c.strip() for c in df_mod.columns]

    if not {"dia", "mes", "ano"}.issubset({c.lower() for c in df_mod.columns}):
        raise ValueError(f"Esperado colunas dia, mes, ano em {arq_mod}")

    col_dia_m = [c for c in df_mod.columns if c.lower() == "dia"][0]
    col_mes_m = [c for c in df_mod.columns if c.lower() == "mes"][0]
    col_ano_m = [c for c in df_mod.columns if c.lower() == "ano"][0]

    df_mod["data"] = pd.to_datetime(
        dict(year=df_mod[col_ano_m], month=df_mod[col_mes_m], day=df_mod[col_dia_m]),
        errors="coerce"
    )
    df_mod = df_mod.dropna(subset=["data"]).set_index("data").sort_index()

    col_modelos = [c for c in df_mod.columns if c not in [col_dia_m, col_mes_m, col_ano_m]]

    # --- Observado ---
    arq_obs = PASTA_OBS / f"{estacao}.csv"
    if not arq_obs.exists():
        raise FileNotFoundError(f"Arquivo de observados não encontrado: {arq_obs}")

    # sem cabeçalho: dia;mes;ano;Qobs
    df_obs = pd.read_csv(arq_obs, sep=';', header=None)
    if df_obs.shape[1] != 4:
        raise ValueError(f"Esperado 4 colunas (dia;mes;ano;Qobs) em {arq_obs}")

    df_obs.columns = ["dia", "mes", "ano", "Qobs"]

    df_obs["data"] = pd.to_datetime(
        dict(year=df_obs["ano"], month=df_obs["mes"], day=df_obs["dia"]),
        errors="coerce"
    )
    df_obs = df_obs.dropna(subset=["data"]).set_index("data").sort_index()
    df_obs = df_obs[["Qobs"]]

    # --- Merge (interseção de datas) ---
    df = df_mod[col_modelos].join(df_obs, how="inner")
    if df.empty:
        raise ValueError(f"Não há datas em comum entre modelos e observado para a estação {estacao}.")

    df = df[["Qobs"] + col_modelos]

    return df, col_modelos

def calcular_medias_mensais(df):
    """
    Médias mensais a partir da série diária.
    Usa 'ME' (MonthEnd).
    """
    mensal = df.resample("ME").mean()
    mensal["mes"] = mensal.index.month
    mensal["ano"] = mensal.index.year
    return mensal

def curva_permanencia(serie, clip_percentis=None):
    s = pd.Series(serie).dropna()

    if clip_percentis is not None:
        lo, hi = clip_percentis
        q_lo = s.quantile(lo)
        q_hi = s.quantile(hi)
        s = s[(s >= q_lo) & (s <= q_hi)]

    if len(s) == 0:
        return np.array([]), np.array([])

    vals = np.sort(s.values)[::-1]
    n = len(vals)
    exced = np.arange(1, n + 1) / (n + 1) * 100.0
    return exced, vals

# ==================== PLOTS: 1 – VIOLIN POR MODELO (ESTILO FIGURA) ============

def plot_violin_mensal_por_modelo(mensal, modelos, nome_estacao, pasta_saida):
    """
    Para cada modelo gera uma figura:
      - 12 violins coloridos (um por mês) das vazões médias mensais do modelo
      - boxplots em preto/branco dentro dos violins
      - pontos da mediana observada deslocados (comparação modelo x obs)
    """
    if mensal.empty:
        print(f"⚠ Mensal vazio em {nome_estacao}; violins por modelo não serão gerados.")
        return

    cmap = plt.get_cmap("tab20")

    for modelo in modelos:
        # dados modelos/obs dessa estação
        dados_modelo = []
        med_obs = []
        for m in range(1, 13):
            vals_mod = mensal.loc[mensal["mes"] == m, modelo].dropna().values
            # garantir pelo menos 2 pontos pra KDE
            if len(vals_mod) == 0:
                vals_mod = np.array([0.001, 0.0011])
            elif len(vals_mod) == 1:
                v = float(vals_mod[0])
                eps = abs(v) * 0.001 if v != 0 else 0.001
                vals_mod = np.array([v - eps, v + eps])
            dados_modelo.append(vals_mod)

            med = mensal.loc[mensal["mes"] == m, "Qobs"].median()
            med_obs.append(med)

        # limites y (modelo + obs)
        all_vals = np.concatenate([np.concatenate(dados_modelo), np.array(med_obs, dtype=float)])
        all_vals = all_vals[np.isfinite(all_vals)]
        if all_vals.size == 0:
            print(f"⚠ Sem dados válidos para modelo {modelo} na estação {nome_estacao}.")
            continue
        vmin = max(np.nanmin(all_vals), 0.01)
        vmax = np.nanmax(all_vals) * 1.1

        fig, ax = plt.subplots(figsize=(9, 4.5))

        pos = np.arange(1, 13)

        # violins
        parts = ax.violinplot(
            dados_modelo,
            positions=pos,
            showmeans=False,
            showextrema=False,
            widths=0.9,
        )

        colors = [cmap(i % 20) for i in range(12)]
        for i, pc in enumerate(parts["bodies"]):
            pc.set_facecolor(colors[i])
            pc.set_edgecolor("none")
            pc.set_alpha(0.7)

        # boxplots dentro dos violins
        bp = ax.boxplot(
            dados_modelo,
            positions=pos,
            widths=0.25,
            patch_artist=True,
            showfliers=False
        )
        for patch in bp["boxes"]:
            patch.set_facecolor("white")
            patch.set_edgecolor("black")
            patch.set_linewidth(1.0)
        for whisker in bp["whiskers"]:
            whisker.set_color("black")
            whisker.set_linewidth(1.0)
        for cap in bp["caps"]:
            cap.set_color("black")
            cap.set_linewidth(1.0)
        for median in bp["medians"]:
            median.set_color("black")
            median.set_linewidth(1.0)

        # mediana observada (ponto preto, deslocado ligeiramente)
        ax.scatter(
            pos + 0.25,
            med_obs,
            s=25,
            c="black",
            zorder=3,
            label="Mediana observada"
        )

        ax.set_xticks(pos)
        ax.set_xticklabels(MES_LABELS)
        ax.set_yscale("log")
        ax.set_ylim(vmin, vmax)
        ax.set_ylabel("Vazão Média Mensal (m³/s)")
        ax.set_title(f"Distribuição da Vazão Média Mensal - Modelo {modelo}\nEstação {nome_estacao}")
        ax.grid(True, which="both", linestyle=":", linewidth=0.5, alpha=0.7)

        ax.legend(loc="upper right")

        fig.tight_layout()
        fig.savefig(
            pasta_saida / f"{nome_estacao}_modelo_{modelo}_violin_mensal.png",
            dpi=300
        )
        plt.close(fig)

# ==================== PLOTS: 2 – VIOLIN ENSEMBLE ==============================

def plot_violin_mensal_ensemble(mensal, modelos, nome_estacao, pasta_saida):
    """
    Distribuição das vazões médias mensais comparando a média dos modelos x observado.
    """
    if mensal.empty:
        print(f"⚠ Mensal vazio em {nome_estacao}; violin ensemble não será gerado.")
        return

    mensal = mensal.copy()
    mensal["ensemble_mean"] = mensal[modelos].mean(axis=1)

    vals = mensal[["Qobs", "ensemble_mean"]].to_numpy().astype(float).ravel()
    vals = vals[np.isfinite(vals)]
    if vals.size == 0:
        print(f"⚠ Sem dados válidos para violin ensemble em {nome_estacao}.")
        return

    vmin = max(np.nanmin(vals), 0.01)
    vmax = np.nanmax(vals) * 1.1

    fig, ax = plt.subplots(figsize=(8, 4.5))

    dados_violin = []
    med_obs = []
    for m in range(1, 13):
        vals_ens = mensal.loc[mensal["mes"] == m, "ensemble_mean"].dropna().values
        if len(vals_ens) == 0:
            vals_ens = np.array([0.001, 0.0011])
        elif len(vals_ens) == 1:
            v = float(vals_ens[0])
            eps = abs(v) * 0.001 if v != 0 else 0.001
            vals_ens = np.array([v - eps, v + eps])
        dados_violin.append(vals_ens)

        med = mensal.loc[mensal["mes"] == m, "Qobs"].median()
        med_obs.append(med)

    pos = np.arange(1, 13)

    parts = ax.violinplot(
        dados_violin,
        positions=pos,
        showmeans=False,
        showextrema=False,
        widths=0.9,
    )

    cmap = plt.get_cmap("tab20")
    for i, pc in enumerate(parts["bodies"]):
        pc.set_facecolor(cmap(i % 20))
        pc.set_edgecolor("none")
        pc.set_alpha(0.7)

    bp = ax.boxplot(
        dados_violin,
        positions=pos,
        widths=0.25,
        patch_artist=True,
        showfliers=False
    )
    for patch in bp["boxes"]:
        patch.set_facecolor("white")
        patch.set_edgecolor("black")
        patch.set_linewidth(1.0)
    for whisker in bp["whiskers"]:
        whisker.set_color("black")
        whisker.set_linewidth(1.0)
    for cap in bp["caps"]:
        cap.set_color("black")
        cap.set_linewidth(1.0)
    for median in bp["medians"]:
        median.set_color("black")
        median.set_linewidth(1.0)

    ax.scatter(
        pos + 0.25,
        med_obs,
        s=25,
        c="black",
        zorder=3,
        label="Mediana observada"
    )

    ax.set_xticks(pos)
    ax.set_xticklabels(MES_LABELS)
    ax.set_yscale("log")
    ax.set_ylim(vmin, vmax)
    ax.set_ylabel("Vazão Média Mensal (m³/s)")
    ax.set_xlabel("Mês")
    ax.set_title(
        f"Distribuição da Vazão Média Mensal\nMédia dos modelos vs Observado - Estação {nome_estacao}"
    )
    ax.legend(loc="upper right")
    ax.grid(True, which="both", linestyle=":", linewidth=0.5, alpha=0.7)

    fig.tight_layout()
    fig.savefig(
        pasta_saida / f"{nome_estacao}_ensemble_violin_mensal.png",
        dpi=300
    )
    plt.close(fig)

# ==================== PLOTS: 3, 4, 5 – FDCs ===================================

def plot_fdc_por_modelo(df, modelos, nome_estacao, pasta_saida,
                        clip_percentis=None, sufixo=""):
    """
    3 e 4) Curvas de permanência diárias para cada modelo comparando com o observado.
    """
    n_models = len(modelos)
    ncols = 4
    nrows = math.ceil(n_models / ncols)

    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 3.5 * nrows),
                             sharex=True, sharey=True)
    axes = axes.ravel()

    x_obs, y_obs = curva_permanencia(df["Qobs"], clip_percentis=clip_percentis)
    cmap = plt.get_cmap("tab20")

    for i, modelo in enumerate(modelos):
        ax = axes[i]
        x_mod, y_mod = curva_permanencia(df[modelo], clip_percentis=clip_percentis)
        if len(x_mod) == 0 or len(x_obs) == 0:
            continue

        ax.plot(x_mod, y_mod, color=cmap(i % 20), linewidth=1.5, label=modelo)
        ax.plot(x_obs, y_obs, color="black", linewidth=1.2, label="Obs")

        ax.set_title(modelo, fontsize=9)
        ax.set_yscale("log")
        if i % ncols == 0:
            ax.set_ylabel("Vazão (m³/s)")
        if i // ncols == nrows - 1:
            ax.set_xlabel("Percentual de Excedência (%)")

    for j in range(n_models, len(axes)):
        fig.delaxes(axes[j])

    if clip_percentis is None:
        extra = ""
    else:
        lo, hi = clip_percentis
        extra = f" (desconsiderando p{int(lo*100)} e p{int(hi*100)})"

    fig.suptitle(f"Curvas de Permanência Diária - Modelos vs Observado\n"
                 f"Estação {nome_estacao}{extra}",
                 fontsize=12)
    fig.tight_layout(rect=[0, 0, 1, 0.93])

    nome_fig = f"{nome_estacao}_3_4_fdc_modelos_vs_obs{sufixo}.png"
    fig.savefig(pasta_saida / nome_fig, dpi=300)
    plt.close(fig)

def plot_fdc_ensemble(df, modelos, nome_estacao, pasta_saida,
                      clip_percentis=None, sufixo=""):
    """
    5) Curva de permanência diária para a média dos modelos comparando com período observado.
    """
    df = df.copy()
    df["ensemble_mean"] = df[modelos].mean(axis=1)

    x_obs, y_obs = curva_permanencia(df["Qobs"], clip_percentis=clip_percentis)
    x_ens, y_ens = curva_permanencia(df["ensemble_mean"], clip_percentis=clip_percentis)

    fig, ax = plt.subplots(figsize=(6, 4))

    ax.plot(x_ens, y_ens, color="tab:blue", linewidth=2, label="Média dos modelos")
    ax.plot(x_obs, y_obs, color="black", linewidth=1.5, label="Obs")

    ax.set_yscale("log")
    ax.set_xlabel("Percentual de Excedência (%)")
    ax.set_ylabel("Vazão (m³/s)")

    if clip_percentis is None:
        extra = ""
    else:
        lo, hi = clip_percentis
        extra = f" (desconsiderando p{int(lo*100)} e p{int(hi*100)})"

    ax.set_title(f"Curva de Permanência Diária - Média dos modelos vs Observado\n"
                 f"Estação {nome_estacao}{extra}")
    ax.legend()
    ax.grid(True, which="both", linestyle=":", linewidth=0.5)

    fig.tight_layout()
    nome_fig = f"{nome_estacao}_5_fdc_ensemble_vs_obs{sufixo}.png"
    fig.savefig(pasta_saida / nome_fig, dpi=300)
    plt.close(fig)

# ==================== LOOP PRINCIPAL ==========================================

for caminho_csv in PASTA_DIARIOS.glob("*.csv"):
    estacao = caminho_csv.stem
    try:
        df, modelos = carregar_dados(estacao)
    except Exception as e:
        print(f"⚠ Erro ao processar estação {estacao}: {e}")
        continue

    print(f"Processando estação {estacao} ({len(modelos)} modelos)...")

    mensal = calcular_medias_mensais(df)

    # 1) violins por modelo (estilo figura enviada)
    plot_violin_mensal_por_modelo(mensal, modelos, estacao, PASTA_SAIDA)

    # 2) violin ensemble x observado
    plot_violin_mensal_ensemble(mensal, modelos, estacao, PASTA_SAIDA)

    # 3 e 4) FDC por modelo
    plot_fdc_por_modelo(df, modelos, estacao, PASTA_SAIDA,
                        clip_percentis=None, sufixo="_completa")

    plot_fdc_por_modelo(df, modelos, estacao, PASTA_SAIDA,
                        clip_percentis=(0.01, 0.99), sufixo="_p1_p99_removidos")

    # 5) FDC ensemble
    plot_fdc_ensemble(df, modelos, estacao, PASTA_SAIDA,
                      clip_percentis=None, sufixo="_completa")
    plot_fdc_ensemble(df, modelos, estacao, PASTA_SAIDA,
                      clip_percentis=(0.01, 0.99), sufixo="_p1_p99_removidos")

    print(f"✔ Figuras geradas para {estacao} em {PASTA_SAIDA}")